In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [7]:
df.columns.tolist()

['user_id',
 'age',
 'status',
 'sex',
 'orientation',
 'body_type',
 'diet',
 'drinks',
 'drugs',
 'education',
 'ethnicity',
 'height',
 'income',
 'job',
 'last_online',
 'location',
 'offspring',
 'pets',
 'religion',
 'sign',
 'smokes',
 'speaks',
 'essay0',
 'essay1',
 'essay2',
 'essay3',
 'essay4',
 'essay5',
 'essay6',
 'essay7',
 'essay8',
 'essay9',
 'bio_text',
 'lat',
 'lon',
 'diet_c',
 'drinks_c',
 'smokes_c',
 'drugs_c']

In [3]:
df.head(5)

,user_id,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay7,essay8,essay9,bio_text,lat,lon,diet_c,drinks_c,smokes_c,drugs_c
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,37.653540,-122.416866,omnivore,light,light,none
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,,i am very open and will share just about anyth...,,i am a chef: this is what that means. 1. i am ...,37.804456,-122.271356,other,heavy,none,sometimes
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te...",37.779259,-122.419329,omnivore,light,none,
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,,,you feel so inclined.,i work in a library and go to school. . . <e> ...,37.870839,-122.272863,vegetarian,light,none,
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,,,,hey how's it going? currently vague on the pro...,37.779259,-122.419329,,light,none,none


In [14]:
import pandas as pd

# Take only the first 10 characters (YYYY-MM-DD)
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)

# Parse as date
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# Now build recency features
df["recency_rank"] = df["last_online_dt"].rank(ascending=True)        # oldest=1
mask = df["last_online_dt"].notna()

# Normalized 0–1 scale within dataset
min_dt, max_dt = df["last_online_dt"].min(), df["last_online_dt"].max()
if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span).fillna(0.0)
else:
    df["recency_norm"] = 0.0

print(df[["last_online_dt","recency_norm"]].head(10))

             last_online_dt  recency_norm
0 2012-06-28 00:00:00+00:00      0.991892
1 2012-06-29 00:00:00+00:00      0.994595
2 2012-06-27 00:00:00+00:00      0.989189
3 2012-06-28 00:00:00+00:00      0.991892
4 2012-06-27 00:00:00+00:00      0.989189
5 2012-06-29 00:00:00+00:00      0.994595
6 2012-06-25 00:00:00+00:00      0.983784
7 2012-06-29 00:00:00+00:00      0.994595
8 2012-06-29 00:00:00+00:00      0.994595
9 2012-06-28 00:00:00+00:00      0.991892


In [25]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [30]:
cands = get_matches(user_id=u,k=500,use_orientation=True,use_location=True, max_km=50,
    use_age=True,use_diet=False,use_drinks=False,use_smokes=False,use_drugs=False,
    drinks_strict=False, smokes_strict=False, drugs_strict=False,allow_missing=True).copy()

# Compute components and map score
cands = compute_component_scores(user_id=u, cands=cands, max_km=50,
    drinks_strict=False, smokes_strict=False, drugs_strict=True,
    allow_missing=True)
cands = _ensure_text_sim(cands)  

cands[["user_id","text_sim","age_score","lifestyle_score","loc_score"]].head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2074
Unfiltered: 2074 | After location filter (≤50 km): 2040
After location (≤50 km): 2040
After age rule: 1126


,user_id,text_sim,age_score,lifestyle_score,loc_score
0,58565,0.664278,1.0,1.000000,1.0
8,43841,0.654167,1.0,1.000000,1.0
22,47209,0.644579,1.0,1.000000,1.0
27,57311,0.642219,1.0,0.666667,1.0
30,49936,0.641363,1.0,0.750000,1.0
31,45885,0.641020,1.0,1.000000,1.0
33,27797,0.640599,1.0,1.000000,1.0
37,15647,0.639417,1.0,1.000000,1.0
41,44908,0.636920,1.0,1.000000,1.0
42,21873,0.636885,1.0,1.000000,1.0
